# 股票价格预测实验笔记本

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pytorch_lightning as pl

from src.data_loader import load_stock_data
from src.preprocess import preprocess_data
from src.feature_engineering import generate_features
from models.lstm_model import StockLSTMModel
from models.transformer_model import StockTransformerModel

In [ ]:
# 数据加载与预处理
raw_data = load_stock_data()
processed_data = preprocess_data(raw_data)
features, labels = generate_features(processed_data)

In [ ]:
# 数据分割
train_size = int(0.8 * len(features))
X_train, X_test = features[:train_size], features[train_size:]
y_train, y_test = labels[:train_size], labels[train_size:]

In [ ]:
# LSTM模型实验
class LSTMExperiment(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = StockLSTMModel(
            input_size=X_train.shape[2],
            hidden_size=64,
            num_layers=2
        )
        self.criterion = nn.MSELoss()
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.model(x)
        loss = self.criterion(y_pred, y)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [ ]:
# 训练LSTM模型
lstm_experiment = LSTMExperiment()
trainer = pl.Trainer(
    max_epochs=50,
    accelerator='gpu',
    devices=1
)
trainer.fit(lstm_experiment)

In [ ]:
# Transformer模型实验
class TransformerExperiment(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = StockTransformerModel(
            input_size=X_train.shape[2],
            d_model=128,
            nhead=4
        )
        self.criterion = nn.MSELoss()
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.model(x)
        loss = self.criterion(y_pred, y)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0005)

In [ ]:
# 训练Transformer模型
transformer_experiment = TransformerExperiment()
trainer = pl.Trainer(
    max_epochs=50,
    accelerator='gpu',
    devices=1
)
trainer.fit(transformer_experiment)

In [ ]:
# 模型性能评估
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        mse = nn.MSELoss()(predictions, y_test)
        return mse.item()